## Running RAJAPerf Problem Size Sweeps

In this section we will investigate the effect of problem size on performance. 
We'll be relying on a couple of built-in scripts that are deployed with RAJAPerf
Namely sweep_size.sh and sweep_graph.py

[Back to Table of Contents](./00-intro-and-contents.ipynb)

## First we create directories to store the data, then initiate sweeps for GCC and Clang
We'll pass in min and max problem sizes increasing by a factor of 2 in size 
We'll also slightly increase the number of reps to 3 to decrease the noise in the data

Note that we'll only be running the kernels in the Stream group for this exercise

Note in practice you would allow RAJAPerf to run at the default number of Repetitions per kernel
and as well increase the number of passes to run the whole suite to capture better min/max/avg
values.

In [ ]:
%%bash
mkdir -p $HOME/data/sweeps && pushd $HOME/data/sweeps
chmod +x $HOME/code/RAJAPerf/scripts/sweep_size.sh
$HOME/code/RAJAPerf/scripts/sweep_size.sh -x sweep_gcc $HOME/code/RAJAPerf/build_gcc/bin/raja-perf.exe --size-min 1024 --size-max 10000000 --size-ratio 2 -- --checkrun 3 --kernels Stream
$HOME/code/RAJAPerf/scripts/sweep_size.sh -x sweep_clang $HOME/code/RAJAPerf/build_clang/bin/raja-perf.exe --size-min 1024 --size-max 10000000 --size-ratio 2 -- --checkrun 3 --kernels Stream

## Generate single line graphs of bandwidth 
The following command will generate a single line graph against all of the problem
sizes generated in the sweep runs, the metric plotted is bandwidth(GB/s) which
relies on the data transfered per unit time. So the Stream benchmarks are a good
set of kernels to evaluate bandwidth for, as they are mostly bandwidth bound.

We're also going to single out the RAJA_OpenMP vs RAJA_Seq variants to plot, and
note that we expect the OpenMP variant to be perform worse or otherwise be erratic
at small problem sizes. So the graph shows where the cutoff might be for a sequential
vs parallel run, or which compiler is best at a particular problem size.

The command will generate .png files with prefix "graph" in the $HOME/data/sweeps 
directory and can be viewed in the browser, by navigating to this directory,
and double clicking on the graph

In [ ]:
%%bash
pushd $HOME/data/sweeps
/opt/conda/bin/python3 $HOME/code/RAJAPerf/scripts/sweep_graph.py  -d sweep_gcc sweep_clang --caliper -v RAJA_Seq RAJA_OpenMP -slg "bandwidth(GB/s)" -o graph